In [ ]:
# module_directory = os.path.join(os.getcwd().split('DSI_Capstone_Steemit')[0],'DSI_Capstone_Steemit')
# sys.path
# sys.path.insert(1,module_directory)

# Need to run Wrangling Notebook Before This One

In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import nltk
from nltk.tokenize import word_tokenize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

data_directory = '../data/'
posts_path = os.path.join(data_directory,'sample_data.csv')
df_posts = pd.read_csv(posts_path)

### Load Data

In [ ]:
posts_counts_path = os.path.join(data_directory,'posts_counts', 'posts_counts')

posts_counts = joblib.load(posts_counts_path)
posts_counts_features = joblib.load(posts_counts_path+'_feature_names')



# Add columns (vocabulary from Vectorizer)
df_posts_counts = pd.DataFrame(posts_counts.toarray())
df_posts_counts.columns = posts_counts_features




In [ ]:
df_posts_counts.shape

### Word2Vec
You need to download google Word2Vec Model

https://docs.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download

In [ ]:
from gensim.models import Word2Vec

In [ ]:
# Load Google's pre-trained Word2Vec model.
word2vec_path = '../word2vec_models/GoogleNews-vectors-negative300.bin'
model = (Word2Vec.
         load_word2vec_format(word2vec_path,
                              binary=True))

In [ ]:
# model.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
# Determine common vocab between Posts Vocabulary and Word2Vec Vocab

s1 = set(model.vocab.keys())
s2 = set(posts_counts_features)

common_vocab = set.intersection(s1, s2)



In [ ]:
len(common_vocab)

In [ ]:
df_posts_word2vec = df_posts_counts.ix[:,common_vocab]

In [ ]:
# Remove any posts that now have word counts of 0
idx_post_remove = (df_posts_word2vec.sum(axis = 1) != 0).values

df_posts_word2vec_desc = df_posts.ix[idx_post_remove,:]
df_posts_word2vec = df_posts_word2vec.ix[idx_post_remove,:]


In [ ]:
number_of_posts = df_posts_word2vec.shape[0]
word2vec_doc_matrix_avg = np.zeros((
                            number_of_posts,
                            300))

for i,row in enumerate(df_posts_word2vec.index.values):
    
    word2vec_temp = np.zeros((
                            len(df_posts_word2vec.columns),
                            300))


    
    # Get all values where they are greater than 0    
    greater_than_0 = (df_posts_word2vec.ix[row,:] > 0).values

    # Get word2vec values that are greater than 0
    vocab = df_posts_word2vec.columns[greater_than_0]
    
    if greater_than_0.sum() > 0:
        word_vector = model[vocab]
        word2vec_temp[greater_than_0,:] = word_vector
        
        mean_doc_vector = word2vec_temp.mean(axis = 0)
        word2vec_doc_matrix_avg[i,:] = mean_doc_vector

Can try using average of Word2Vec to get Doc2Vec, or Multiply it by TFIDF then average

http://stackoverflow.com/questions/29760935/how-to-get-vector-for-a-sentence-from-the-word2vec-of-tokens-in-sentence

In [ ]:
word2vec_doc_matrix_avg_path = os.path.join(data_directory,'word2vec_doc_matrix_avg', 'word2vec_doc_matrix_avg')
joblib.dump(word2vec_doc_matrix_avg,word2vec_doc_matrix_avg_path)